# Calculation of ground state energies of the Z2 LGT Hamiltonian

## Sign convention

The $J$ and $f$ parameters of the Z2 LGT Hamiltonian can appear in both signs, but I confirmed numerically that all sign combinations lead to the same spectrum. Would be good to give an analytical explanation as to why.

## Strategy

Given a computational basis state $\ket{k}$, $\ket{\tilde{k}} = \otimes_{n=0}^{N_s-1} H_{2n+1} \ket{k}$ is an eigenstate of the Gauss' law operators $G_n = -X_{2n-1} Z_{2n} X_{2n+1}$. The set $\{\ket{\tilde{k}}\}$ forms an orthonormal basis of the full Hilbert space and divides it into subspaces defined by Gauss' law and $U(1)$ eigenvalues. The Hamiltonian is block diagonal in this basis, i.e., its matrix element $\bra{\tilde{k}} H \ket{\tilde{l}}$ is nonzero only if $\ket{\tilde{k}}$ and $\ket{\tilde{l}}$ belong to the same symmetry sector. To identify the ground state energy reached by an ideal VQE run, we diagonalize the symmetry block that the initial state belongs to.

In [1]:
import sys
import os
sys.path.append('z2vqe/src')
import numpy as np
import scipy
import jax
import jax.numpy as jnp
from qiskit.quantum_info import SparsePauliOp

from z2_lgt import calculate_num_params, z2_ansatz_layer, create_hamiltonian, initial_state

In [2]:
num_sites = 8
j_hopping = 1  # coupling constant J
f_gauge = 1 / 2  # coupling constant f
mass = 2  # value of mass

hamiltonian = create_hamiltonian(num_sites, j_hopping, f_gauge, mass, 0., 'closed', False).to_matrix(sparse=True)
hamiltonian = scipy.sparse.csr_array(hamiltonian)
hamiltonian.data = hamiltonian.data.real

In [3]:
def get_symmetry_sector(state):
    num_qubits = np.log2(state.shape[0]).astype(int)
    num_sites = num_qubits // 2
    state = state.reshape((2,) * (2 * num_sites))
    hadamard = np.array([[1., 1.], [1., -1.]]) / np.sqrt(2.)
    for isite in range(num_sites):
        state = np.moveaxis(np.tensordot(hadamard, state, [1, isite * 2 + 1]), 0, isite * 2 + 1)

    zeigvals = np.empty(num_qubits, dtype=int)
    for iqubit in range(num_qubits):
        moved = np.moveaxis(state, iqubit, 0)
        if np.allclose(moved[0], 0.):
            zeigvals[iqubit] = -1
        elif np.allclose(moved[1], 0.):
            zeigvals[iqubit] = 1
        else:
            raise ValueError('Not a Gauss law eigenstate')

    gauss_sector = np.roll(zeigvals, -1)[::2] * zeigvals[::2] * np.roll(zeigvals, 1)[::2]
    u1_sector = np.sum(zeigvals[::2]) / num_sites
    return gauss_sector, u1_sector

In [4]:
num_sites = 8
num_qubits = 2 * num_sites
init_state = np.zeros((2,) * num_qubits)
init_state[(0,) * num_qubits] = 1.
paulix = np.array([[0., 1.], [1., 0.]])
hadamard = np.array([[1., 1.], [1., -1.]]) / np.sqrt(2.)
for iq in range(0, num_qubits, 4):
    init_state = np.moveaxis(np.tensordot(paulix, init_state, [1, iq]), 0, iq)
for iq in range(1, num_qubits, 2):
    init_state = np.moveaxis(np.tensordot(hadamard, init_state, [1, iq]), 0, iq)

gauss_sector, u1_sector = get_symmetry_sector(init_state.reshape(-1))

In [5]:
state_zeigvals = 1 - 2 * ((np.arange(2 ** num_qubits)[:, None] >> np.arange(num_qubits)[None, ::-1]) % 2)
state_gauss = np.roll(state_zeigvals, -1, axis=1)[:, ::2] * state_zeigvals[:, ::2] * np.roll(state_zeigvals, 1, axis=1)[:, ::2]
state_u1 = np.sum(state_zeigvals[:, ::2], axis=1) / num_sites
subspace = np.nonzero(np.all(np.equal(state_gauss, gauss_sector[None, :]), axis=1) & np.equal(state_u1, u1_sector))[0]

In [6]:
# One-hot vectors
subbasis = np.zeros((2 ** num_qubits,) + subspace.shape)
subbasis[subspace, np.arange(subspace.shape[0])] = 1.
subbasis = subbasis.reshape((2,) * num_qubits + subspace.shape)
for isite in range(num_sites):
    subbasis = np.moveaxis(np.tensordot(hadamard, subbasis, [1, 2 * isite + 1]), 0, 2 * isite + 1)
subbasis = scipy.sparse.csr_array(subbasis.reshape((2 ** num_qubits,) + subspace.shape))

In [7]:
block_hamiltonian = (subbasis.T @ (hamiltonian @ subbasis)).todense()
block_hamiltonian = np.where(np.isclose(block_hamiltonian, 0.), 0., block_hamiltonian)

In [8]:
evals, evecs = np.linalg.eigh(block_hamiltonian)

In [9]:
evals

array([-1.34459900e+01, -8.14786791e+00, -8.00750381e+00, -8.00750381e+00,
       -7.72318362e+00, -7.72318362e+00, -7.52457967e+00, -7.52457967e+00,
       -7.47080983e+00, -6.13660070e+00, -5.82640682e+00, -5.81925351e+00,
       -5.81925351e+00, -5.81351144e+00, -5.81351144e+00, -5.80844009e+00,
       -5.80844009e+00, -5.79576175e+00, -4.01779687e+00, -3.98553959e+00,
       -3.98553959e+00, -3.98499413e+00, -3.98499413e+00, -3.97030985e+00,
       -3.97030985e+00, -3.93759538e+00, -2.66572040e+00, -2.45614612e+00,
       -2.45614612e+00, -2.45401451e+00, -2.37266276e+00, -2.37266276e+00,
       -2.29048458e+00, -2.29048458e+00, -2.15602075e+00, -2.15602075e+00,
       -2.09804465e+00, -2.06937554e+00, -2.06937554e+00, -2.00000000e+00,
       -1.99064518e+00, -1.99064518e+00, -1.93789849e+00, -1.93271167e+00,
       -1.87355975e+00, -1.87355975e+00, -1.70025080e+00, -1.70025080e+00,
       -1.66012673e+00, -1.66012673e+00, -1.61947549e+00, -1.61947549e+00,
       -1.53917425e+00, -